# Practica SPAM Procesamiento del lenguaje natural 

# Alumno Paul Ivan Trujillo Chavez

Los correos electrónicos no deseados en su bandeja de entrada son molestos ya que perturban la rutina del usuarios. Es por eso que las ceuntas de correo electrónico ya tiene un filtro de spam. Dado que e suna de las aplicaciones PLN más utilizadas vamos a ver como se desarrolló un filtro de spam simple para correos electrónicos.

In [1]:
#Importamos las famosas librerías
from functools import reduce

import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
import string
import re

In [2]:
#Insertando los datos
full_corpus = pd.read_csv('SMSSpamCollection.tsv', sep='\t', header=None, names=['label', 'msg_body'])

#Separando los mensajes en 'ham' y 'spam'
ham_text = []
spam_text = []

### Biagrams

Los N-gramos se usan para moldear el lenguaje en función de la predicción de palabras, es decir, predice la siguiente palabra de una oración de palabras de una oración de palabras N-1 anteriores. Bigram es la secuencia de 2 palabras de N-gramos que predice la siguiente palabra de una oración usando la palabra anterior. En lugar de considerar la historia completade una oración o una secuencia de palabras en particular, un modelo como bigram puede ser ocupado en términos de una aproximación de la historia al ocupar una historia limitada.

La identificación de un mensaje como "ham" o "spam" es una tarea de clasificación ya que la variable de destino tiene valores discretos que son "ham"o "spam". En esta práctica, se usa el modelo biagram, aunque existen muchas técnicas avanzadas que se pueden utilizar para este propósito. Para utilizar el modelo bigram para asignar un mensaje dado como "spam" o "ham", hay varios pasos que deben lograrse.

### 1. Inspección y separación de mensajes en las categorías "ham" y "spam"

inicialmente, el conjunto de datos debe inspeccionarse para ocuparlo y abordarlo para lograr la tarea. El formato de los datos dados la cantidad de datos proporcionados, la naturaleza de los datos se incluyen en esta inspección para identificar la mejor aproximación posible para la tarea.

El corpus de mensajes dado ha marcado cada mensaje como ham o spam. Además, hay 5568 mensajes en un DataFrame escrito en inglés que no son objetos nulos. Por lo tanto, el archivo tsv se puede leer usando DataFrame en phyton para clasificar esos mensajes de acuerdo con el indicador dado. 

In [3]:
def separate_msgs():
    for index, column in full_corpus.iterrows():
        label = column[0]
        message_text = column[1]
        if label == 'ham':
            ham_text.append(message_text)
        elif label == 'spam':
            spam_text.append(message_text)

separate_msgs()

### 2. Procesamiento de texto

El procesamiento es la tarea de realizar los pasos de preparación en el corpus de texto sin formato para completar de manera eficiente una extracción de texto o procesamiento de leguaje natural o cualquier otra tarea que incluya texto sin formato. El preprocesaiento de texto consta de varios pasos, aunque algunos de ellos pueden no aplicarse a una tarea en particular debido a la naturaleza del conjunto de datos disponible.

En esta tarea, el procesamiento de texto incluye los siguientes pasos de acuerdo con el conjunto de datos:

### Eliminación de signos de puntuación

In [4]:
#Eliminación de los signos de puntuación de los mensajes de correo electrónico
def remove_msg_punctuations(email_msg):
    puntuation_remove_msg = "".join([word for word in email_msg if word not in string.punctuation])
    return puntuation_removed_msg

### Convertir a minúsculas:

Convertir a minúsculas: La conversión de todos los caracteres del texto en un contexto común, como los soportes en minúsculas, impide identificar dos palabras de madera diferente donde una está en minuscula y la otra no. Por ejemplo, "primero" y "Primero" deben identificarse como iguales, por lo tanto, poner en minúscula todos los caracteres facilita la tarea. Ademas, las palabras de detención también están en minúsculas, por lo que esto también haría posible eliminar palabras de detención más adelante.

### Tokenizing

Tokenizing: Es la tarea de dividir el texto en partes significativas, es decir, tokens que incluyen oraciones y palabras. Un token se puede considerar como una instancia de una secuencia de carcateres en un texto particular que se agrupan para proporcionar una unidad semánitca útil para su posterior procesamiento. En esta tarea, la tokenización de palabras se realiza combinando espacios en blanco entre palabras como delimitador. Esto se logra ...

In [5]:
#Convierte el texto en minusculas y tokenizing de palabras
def tokenize_into_words(text):
    tokens = re.split('\W+', text)
    return tokens

### Plabras lematizantes

La derivación es el proceso de eliminar (sufijos, prefijos, infijos, circunfijos) de una palabra para obtener su raíz de palabra. Aunque la lematización está relacionada con la derivación, difiere ya que lematización puede capturar formas canónicas basadas en el tema de una palabra. La lematización ocupa un vocabulario y un análisis morfológico de las palabras que los hacan más rápido y preciso que la derivación. WordNetLemmatizer ha logrado la lematización en lenguaje Phyton.

In [7]:
#Lemmatizing
word_lemmatizer = WordNetLemmatizer()
def lemmatization(tokenized_words):
    lemmatized_text = [word_lemmatizer.lemmatize(word)for word in tokenized_words]
    return ' '.join(lemmatized_text)

def preprocessing_msgs(corpus):
    categorized_text = pd.DataFrame(corpus)
    categorized_text['non_punc_message_body'] = categorized_text[0].apply(lambda msg: remove_msg_punctuation(msg))
    categorized_text['tokenized_msg_body'] = categorized_text['non_punc_message_body'].apply(lambda msg: tokenize_into_words(msg.lower()))
    categorized_text['lemmatized_msg_words'] = categorized_text['tokenized_msg_body'].apply(lambda word_list: lemmatization(word_list))
    return categorized_text['lemmatized_msg_words']

### 3. Extracción de características

Después de la etapa  de procesamiento, las caracteristicas deben extraerse del texto. Las caracteristicas son las unidades que admiten la tarea de clasificación, y las bigrams son las caracteristicas en esta tarea de clasificación de mensajes. Los birams o las caracteristicas se extraen dek texto preprocesado Inicialmente, los unigramas se adquieren, y luego esos unigramas se utilizan para obtener los unigramas en cada corpus ("ham" y "spam").